In [9]:
pip install transformers datasets torch scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
pip install evaluate

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
pip install seqeval


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for sentencepiece from https://files.pythonhosted.org/packages/a2/f6/587c62fd21fc988555b85351f50bbde43a51524caafd63bc69240ded14fd/sentencepiece-0.2.0-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   ------------ --------------------------- 307.2/991.5 kB 6.3 MB/s eta 0:00:01
   --------------------------------------  983.0/991.5 kB 12.5 MB/s eta 0:00:01
   ---------------------------------------- 991.5/991.5 kB 9.0 MB/s eta 0:00:00


In [10]:
from datasets import load_dataset
dataset = load_dataset("ai4bharat/naamapadam", "kn")

In [12]:
dataset["train"].features["tokens"]
dataset["train"][0]

{'tokens': ['ನಿರ್ದೇಶಕರುಃ', 'ಶಿವಗಣೇಶ್'], 'ner_tags': [0, 1]}

In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [8]:

# (Optional) Example text in Indic languages
example_text = "ಶಿವಗಣೇಶ್"  # Replace with relevant text for testing
inputs = tokenizer(example_text, return_tensors="pt").to(device)

# Forward pass (example usage)
outputs = model(**inputs)
outputs

TokenClassifierOutput(loss=None, logits=tensor([[[-0.0597, -0.0920],
         [ 0.0966,  0.2379],
         [ 0.0473,  0.2500],
         [-0.0885,  0.2993],
         [-0.0597, -0.0920]]], grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [ ]:
# Import necessary libraries
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from datasets import load_dataset
import evaluate
import numpy as np
import torch

# Check for GPU availability and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the dataset
dataset = load_dataset("ai4bharat/naamapadam", "kn")

# Load evaluation metric
metric = evaluate.load("seqeval")

# Load pre-trained tokenizer and model
model_name = "ai4bharat/indic-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name).to(device)

# Preprocessing function
label_list = dataset["train"].features["ner_tags"].feature.names

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        max_length=128,
        padding="max_length"
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        aligned_labels = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                aligned_labels.append(-100)
            elif word_idx != previous_word_idx:
                aligned_labels.append(label[word_idx])
            else:
                aligned_labels.append(-100)
            previous_word_idx = word_idx
        labels.append(aligned_labels)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Preprocess datasets
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

# Define metric computation
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Training arguments
args = TrainingArguments(
    "test-ner",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
    logging_dir='./logs',
    logging_steps=10,
    report_to="none"
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(results)

# Save the model
model.save_pretrained("./ner_model")
tokenizer.save_pretrained("./ner_model")


Using device: cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.70k [00:00<?, ?B/s]

conllpp.py:   0%|          | 0.00/8.73k [00:00<?, ?B/s]

The repository for conllpp contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conllpp.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


...

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-4-26803f18f9af>:94: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.010300,0.039807,0.954102,0.956031,0.955066,0.992128
2,0.000500,0.042127,0.956245,0.960916,0.958575,0.992654
3,0.000600,0.044339,0.956340,0.963106,0.959711,0.992810


{'eval_loss': 0.04433931037783623, 'eval_precision': 0.956339913014386, 'eval_recall': 0.9631064690026954, 'eval_f1': 0.9597112640590901, 'eval_accuracy': 0.9928102410225434, 'eval_runtime': 83.3796, 'eval_samples_per_second': 38.978, 'eval_steps_per_second': 2.447, 'epoch': 3.0}


('./ner_model/tokenizer_config.json',
 './ner_model/special_tokens_map.json',
 './ner_model/vocab.txt',
 './ner_model/added_tokens.json',
 './ner_model/tokenizer.json')

In [7]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
import numpy as np

# Load the saved model and tokenizer
model_path = "./model/ner_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForTokenClassification.from_pretrained(model_path)

# Define the label list (same as during training)
# label_list = ["O", "B-PER", "I-PER", "B-MISC", "I-MISC", "B-LOC", "I-LOC", "B-ORG", "I-ORG"]
label_list = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

# Function for inference
def predict_ner(sentence):
    # Tokenize the input sentence
    tokens = tokenizer(
        sentence,
        truncation=True,
        # is_split_into_words=True,
        return_tensors="pt"
    )

    # Move tokens to the same device as the model
    tokens = {key: value.to(model.device) for key, value in tokens.items()}

    # Get predictions
    with torch.no_grad():
        outputs = model(**tokens)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=2)

    # Decode predictions
    tokens_ids = tokens["input_ids"][0]
    tokens_list = tokenizer.convert_ids_to_tokens(tokens_ids)
    predictions = predictions[0].cpu().numpy()

    results = []
    for token, prediction in zip(tokens_list, predictions):
        label = label_list[prediction]
        results.append((token, label))

    return results

# Example input sentence
sentence = "Barack Obama, The President of US was born in Hawaii."
sentence = "The CIA is an intelligence agency of the US."
predictions = predict_ner(sentence)

# Clean the output to remove special tokens
cleaned_predictions = [
    (token, label) for token, label in predictions if token not in ["[CLS]", "[SEP]", "[PAD]"]
]

# Print predictions
for token, label in cleaned_predictions:
    print(f"{token:15} -> {label}")


The             -> O
CIA             -> B-ORG
is              -> O
an              -> O
intelligence    -> O
agency          -> O
of              -> O
the             -> O
US              -> B-LOC
.               -> O


In [1]:
from datasets import load_dataset
dataset = load_dataset("conllpp", trust_remote_code=True)

d:\Read!!!\Sem 5\CS626\Course Project\Final\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [29]:
dataset["train"].features['ner_tags'].feature.names
dataset["train"].features['ner_tags']


Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [8]:
!zip -r all_files.zip ./*

  adding: ner_model/ (stored 0%)
  adding: ner_model/tokenizer_config.json (deflated 76%)
  adding: ner_model/special_tokens_map.json (deflated 42%)
  adding: ner_model/tokenizer.json (deflated 70%)
  adding: ner_model/model.safetensors (deflated 7%)
  adding: ner_model/config.json (deflated 55%)
  adding: ner_model/vocab.txt (deflated 49%)
  adding: sample_data/ (stored 0%)
  adding: sample_data/README.md (deflated 39%)
  adding: sample_data/anscombe.json (deflated 83%)
  adding: sample_data/mnist_test.csv (deflated 88%)
  adding: sample_data/mnist_train_small.csv (deflated 88%)
  adding: sample_data/california_housing_test.csv (deflated 76%)
  adding: sample_data/california_housing_train.csv (deflated 79%)
  adding: test-ner/ (stored 0%)
  adding: test-ner/checkpoint-2634/ (stored 0%)
  adding: test-ner/checkpoint-2634/rng_state.pth (deflated 25%)
  adding: test-ner/checkpoint-2634/tokenizer_config.json (deflated 76%)
  adding: test-ner/checkpoint-2634/special_tokens_map.json (deflat

In [12]:
from google.colab import files
files.download("all_files.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
from google.colab import drive
drive.mount('/content/drive')

# Copy the model to your Google Drive
!cp -r ./all_files.zip /content/drive/MyDrive/

Mounted at /content/drive


In [11]:
dataseto = load_dataset("ontonotes/conll2012_ontonotesv5",'english_v4', trust_remote_code=True) 

Generating test split: 100%|██████████| 222/222 [00:02<00:00, 84.16 examples/s] 


In [28]:
dataseto["train"].features['sentences'][0]['named_entities']

Sequence(feature=ClassLabel(names=['O', 'B-PERSON', 'I-PERSON', 'B-NORP', 'I-NORP', 'B-FAC', 'I-FAC', 'B-ORG', 'I-ORG', 'B-GPE', 'I-GPE', 'B-LOC', 'I-LOC', 'B-PRODUCT', 'I-PRODUCT', 'B-DATE', 'I-DATE', 'B-TIME', 'I-TIME', 'B-PERCENT', 'I-PERCENT', 'B-MONEY', 'I-MONEY', 'B-QUANTITY', 'I-QUANTITY', 'B-ORDINAL', 'I-ORDINAL', 'B-CARDINAL', 'I-CARDINAL', 'B-EVENT', 'I-EVENT', 'B-WORK_OF_ART', 'I-WORK_OF_ART', 'B-LAW', 'I-LAW', 'B-LANGUAGE', 'I-LANGUAGE'], id=None), length=-1, id=None)

In [31]:
dataset["train"]

Dataset({
    features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
    num_rows: 14041
})

In [34]:
dataseto["train"][0]

{'document_id': 'bc/cctv/00/cctv_0001',
 'sentences': [{'part_id': 0,
   'words': ['What', 'kind', 'of', 'memory', '?'],
   'pos_tags': [46, 24, 17, 24, 7],
   'parse_tree': '(TOP(SBARQ(WHNP(WHNP (WP What)  (NN kind) )(PP (IN of) (NP (NN memory) ))) (. ?) ))',
   'predicate_lemmas': [None, None, None, 'memory', None],
   'predicate_framenet_ids': [None, None, None, None, None],
   'word_senses': [None, None, None, 1.0, None],
   'speaker': 'Speaker#1',
   'named_entities': [0, 0, 0, 0, 0],
   'srl_frames': [],
   'coref_spans': []},
  {'part_id': 0,
   'words': ['We',
    'respectfully',
    'invite',
    'you',
    'to',
    'watch',
    'a',
    'special',
    'edition',
    'of',
    'Across',
    'China',
    '.'],
   'pos_tags': [30, 32, 39, 30, 37, 39, 13, 18, 24, 17, 25, 25, 7],
   'parse_tree': '(TOP(S(NP (PRP We) )(ADVP (RB respectfully) )(VP (VB invite) (NP (PRP you) )(S(VP (TO to) (VP (VB watch) (NP(NP (DT a)  (JJ special)  (NN edition) )(PP (IN of) (NP (NNP Across)  (NNP Ch

In [13]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
from datasets import load_dataset
import evaluate
import numpy as np
import torch

# Check for GPU availability and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the dataset
dataset = load_dataset("ai4bharat/naamapadam", "kn")

# Load evaluation metric
metric = evaluate.load("seqeval")

# Load pre-trained tokenizer and model
model_name = "ai4bharat/indic-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(dataset["train"].features["ner_tags"].feature.names)).to(device)

# Get the label list
label_list = dataset["train"].features["ner_tags"].feature.names

# Preprocessing function
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        max_length=128,
        padding="max_length"
    )

    labels = []
    for i, ner_tags in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        aligned_labels = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                aligned_labels.append(-100)
            elif word_idx != previous_word_idx:
                aligned_labels.append(ner_tags[word_idx])
            else:
                aligned_labels.append(-100)
            previous_word_idx = word_idx
        labels.append(aligned_labels)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Preprocess datasets
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

# Define metric computation
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# Training arguments
args = TrainingArguments(
    "test-ner",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
    logging_dir='./logs',
    logging_steps=10,
    report_to="none"
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(results)

# Save the model
model.save_pretrained("./ner_kn_model")
tokenizer.save_pretrained("./ner_kn_model")


Using device: cpu


Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at ai4bharat/indic-bert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/471763 [00:00<?, ? examples/s]

Map:   0%|          | 0/1019 [00:00<?, ? examples/s]

Map:   0%|          | 0/2381 [00:00<?, ? examples/s]

d:\Read!!!\Sem 5\CS626\Course Project\Final\.venv\Lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>={ACCELERATE_MIN_VERSION}'`